## Predicting Chronic Kidney Disease in Patients

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [43]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

In [15]:
df = pd.read_csv("chronic_kidney_disease_full.csv")

In [16]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [17]:
df.shape

(400, 25)

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer:
we want to predict ckd, I would probably want to compare each variable with ckd to see which variables appear to have predictive value.
- erros, missing and non values
- distribution
- regression model

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [18]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

some data is missing and we have to take care of this.

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [19]:
df.dropna(axis = 0, how = 'any', inplace = False).shape

(158, 25)

In [20]:
df.dropna(axis = 0, how = 'any', inplace = False)['class'].value_counts()

notckd    115
ckd        43
Name: class, dtype: int64

Answer: we would drop lot of data which is not good model.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: Maybe lab results were mixed up (i.e. not connected to the correct patient).
Maybe machines aren't properly calibrated or some contamination occurred so our measurements are incorrect.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: simple model would tell all people that they have CKD. Thus, no person would ever be told they do not have CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer:In this problem, assuming that CKD is the "positive" class, we minimized false negatives. This is equivalent to maximizing sensitivity.

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: Telling every person they have CKD is unethical because then people will likely pursue expensive treatment options

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer:simple model would tell all people that they have CKD. Thus, no person would ever be told they do not have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: specificity

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer:Telling every person they have CKD is unethical because then people will return home thinking they are healthy when they are not. 

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [21]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [24]:
df['y'] = [1 if i == 'ckd' else 0 for i in df['class']]

In [25]:
df['y'].value_counts()

1    250
0    150
Name: y, dtype: int64

In [26]:
df['pc_pcc_interaction'] = [1 if df.loc[i, 'pc'] == 'abnormal' and df.loc[i, 'pcc'] == 'present' else 0 for i in range(len(df['pc']))]

In [27]:
df['wbcc_rbcc_interaction'] = df['wbcc'] * df['rbcc']

In [28]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,htn,dm,cad,appet,pe,ane,class,y,pc_pcc_interaction,wbcc_rbcc_interaction
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,yes,yes,no,good,no,no,ckd,1,0,40560.0
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,no,no,no,good,no,no,ckd,1,0,NaN
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,no,yes,no,poor,no,yes,ckd,1,0,NaN
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,yes,no,no,poor,yes,yes,ckd,1,1,26130.0
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,no,no,no,good,no,no,ckd,1,0,33580.0


In [29]:
rbc_abnormal = pd.get_dummies(df['rbc'])['abnormal']

In [30]:
pc_abnormal = pd.get_dummies(df['pc'])['abnormal']

In [31]:
pcc_present = pd.get_dummies(df['pcc'])['present']

In [32]:
ba_present = pd.get_dummies(df['ba'])['present']

In [33]:
htn_yes = pd.get_dummies(df['htn'])['yes']

In [34]:
dm_yes = pd.get_dummies(df['dm'])['yes']

In [35]:
cad_yes = pd.get_dummies(df['cad'])['yes']

In [36]:
appet_poor = pd.get_dummies(df['appet'])['poor']

In [37]:
pe_yes = pd.get_dummies(df['pe'])['yes']

In [38]:
ane_yes = pd.get_dummies(df['ane'])['yes']

In [39]:
quant = df[['age', 'bp', 'sg', 'al', 'su', 'bgr',
                 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv',
                 'wbcc', 'rbcc', 'pc_pcc_interaction',
                 'wbcc_rbcc_interaction']]

qual = pd.DataFrame([ane_yes, pe_yes, rbc_abnormal, pc_abnormal,
                     pcc_present, ba_present, htn_yes, dm_yes,
                     cad_yes, appet_poor], index=['ane_yes', 'pe_yes', 'rbc_abnormal',
                                                  'pc_abnormal', 'pcc_present',
                                                  'ba_present', 'htn_yes', 'dm_yes',
                                                  'cad_yes', 'appet_poor']).T

In [40]:
X = quant.merge(right = qual, left_index = True, right_index = True)

In [41]:
X.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,ane_yes,pe_yes,rbc_abnormal,pc_abnormal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_poor
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,...,0,0,0,0,0,0,1,1,0,0
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,...,1,0,0,0,0,0,0,1,0,1
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,1,1,0,1,1,0,1,0,0,1
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X.fillna(X.mean()),
                                                    df['y'],
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [45]:
logit = LogisticRegression(C = 1,
                           class_weight = 'balanced',
                           penalty = 'l2',
                           random_state = 42)

In [46]:
logit.fit(X = X_train,
          y = y_train)

/Users/tahir/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, class_weight='balanced', random_state=42)

In [47]:
logit.score(X_train, y_train)

0.9107142857142857

In [48]:
logit.score(X_test, y_test)

0.9166666666666666

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [49]:
list(zip(np.exp(logit.coef_[0]),X.columns))

[(0.9975366089606312, 'age'),
 (1.1214238469465996, 'bp'),
 (0.9999228959309502, 'sg'),
 (1.0308598603942818, 'al'),
 (1.0052745984328189, 'su'),
 (1.046897335733306, 'bgr'),
 (1.0296763935462132, 'bu'),
 (1.0262645384736828, 'sc'),
 (0.9243580501414835, 'sod'),
 (1.0012858133140607, 'pot'),
 (0.9391688410742555, 'hemo'),
 (0.8529128816529268, 'pcv'),
 (1.0008349440497166, 'wbcc'),
 (0.9926511085093435, 'rbcc'),
 (1.0006556373074718, 'pc_pcc_interaction'),
 (0.9999029096403523, 'wbcc_rbcc_interaction'),
 (1.0027091321074886, 'ane_yes'),
 (1.0055596289869089, 'pe_yes'),
 (1.003424390231121, 'rbc_abnormal'),
 (1.0027842365190058, 'pc_abnormal'),
 (1.0016300872309447, 'pcc_present'),
 (1.0010385497030871, 'ba_present'),
 (1.0080216833534414, 'htn_yes'),
 (1.0066492133312281, 'dm_yes'),
 (1.0014421361942174, 'cad_yes'),
 (1.004737911110987, 'appet_poor')]

an individual is 4.05 times as likely to have CKD, all else held equal

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [50]:
list(zip(np.exp(logit.coef_[0]),X.columns))

[(0.9975366089606312, 'age'),
 (1.1214238469465996, 'bp'),
 (0.9999228959309502, 'sg'),
 (1.0308598603942818, 'al'),
 (1.0052745984328189, 'su'),
 (1.046897335733306, 'bgr'),
 (1.0296763935462132, 'bu'),
 (1.0262645384736828, 'sc'),
 (0.9243580501414835, 'sod'),
 (1.0012858133140607, 'pot'),
 (0.9391688410742555, 'hemo'),
 (0.8529128816529268, 'pcv'),
 (1.0008349440497166, 'wbcc'),
 (0.9926511085093435, 'rbcc'),
 (1.0006556373074718, 'pc_pcc_interaction'),
 (0.9999029096403523, 'wbcc_rbcc_interaction'),
 (1.0027091321074886, 'ane_yes'),
 (1.0055596289869089, 'pe_yes'),
 (1.003424390231121, 'rbc_abnormal'),
 (1.0027842365190058, 'pc_abnormal'),
 (1.0016300872309447, 'pcc_present'),
 (1.0010385497030871, 'ba_present'),
 (1.0080216833534414, 'htn_yes'),
 (1.0066492133312281, 'dm_yes'),
 (1.0014421361942174, 'cad_yes'),
 (1.004737911110987, 'appet_poor')]

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer:
Logistic regression allows for interpretable coefficients so that we can understand how X affects y, as we saw above.
Logistic regression usually does not suffer from high variance due to the large number of simplifying assumptions placed on the model. (i.e. features are "linear in the logit," errors are independent and follow a Bernoulli distribution, etc.)

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [51]:
y_preds = logit.predict(X_test)

In [52]:
print(classification_report(y_test,
                            logit.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89        44
           1       0.97      0.89      0.93        76

    accuracy                           0.92       120
   macro avg       0.91      0.92      0.91       120
weighted avg       0.92      0.92      0.92       120



In [53]:
confusion_matrix(y_test, y_preds)

array([[42,  2],
       [ 8, 68]])

In [54]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()

In [55]:
print("True Negatives: " + str(tn))
print()
print("False Positives: " + str(fp))
print()
print("False Negatives: " + str(fn))
print()
print("True Positives: " + str(tp))

True Negatives: 42

False Positives: 2

False Negatives: 8

True Positives: 68


In [56]:
73/(73+3)

0.9605263157894737

### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: I don't think sensitivity and specificity are equally important, though. I think sensitivity is more important

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer:
Unbalanced classes are usually a problem because our model has a tough time learning about the minority class. There's simply not enough data to learn the "pattern" or "behavior" of the minority class.
They probably aren't a problem in this CKD analysis. A 40/60 split isn't substantially unbalanced. However, in gridsearching over whether or not to balance our classes, GridSearchCV found that our model performs better when we balance our y classes.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer:That is the problem of unbalanced classes. When we build a model where we have lots of broken arms but very few "rare diseases," it's hard for our model to learn about those rare diseases because we see them so infrequently that we don't have enough information.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [57]:
ckd_sample = df[df['class'] == 'ckd'].sample(200_000,            # sample n = 200,000
                                               replace = True,     # sample with replacement
                                               random_state = 42)  # set random state

In [58]:
df_2 = pd.concat([df, ckd_sample])

In [59]:
df_2.shape

(200400, 28)

In [60]:
df_2['class'].value_counts() / len(df_2['class'])

ckd       0.999251
notckd    0.000749
Name: class, dtype: float64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [62]:
quant = df_2[['age', 'bp', 'sg', 'al', 'su', 'bgr',
                 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv',
                 'wbcc', 'rbcc', 'pc_pcc_interaction',
                 'wbcc_rbcc_interaction', 'y']]

quant.loc[:,'rbc_abnormal_2'] = pd.get_dummies(df_2['rbc'])['abnormal']
quant.loc[:,'pc_abnormal_2'] = pd.get_dummies(df_2['pc'])['abnormal']
quant.loc[:,'pcc_present_2'] = pd.get_dummies(df_2['pcc'])['present']
quant.loc[:,'ba_present_2'] = pd.get_dummies(df_2['ba'])['present']
quant.loc[:,'htn_yes_2'] = pd.get_dummies(df_2['htn'])['yes']
quant.loc[:,'dm_yes_2'] = pd.get_dummies(df_2['dm'])['yes']
quant.loc[:,'cad_yes_2'] = pd.get_dummies(df_2['cad'])['yes']
quant.loc[:,'appet_poor_2'] = pd.get_dummies(df_2['appet'])['poor']
quant.loc[:,'pe_yes_2'] = pd.get_dummies(df_2['pe'])['yes']
quant.loc[:,'ane_yes_2'] = pd.get_dummies(df_2['ane'])['yes']

/Users/tahir/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/tahir/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [63]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(quant.fillna(quant.mean()).drop(['y'],
                                                                                    axis = 1),
                                                    quant['y'],
                                                    test_size = 0.23, 
                                                    random_state = 42)

In [64]:
logit_2 = LogisticRegression(C = 1,
                             random_state = 42,
                             penalty = 'l2'
                            )

In [65]:
logit_2.fit(X_train_2, y_train_2)

/Users/tahir/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, random_state=42)

In [66]:
print(classification_report(y_test_2,
                            logit_2.predict(X_test_2)))

              precision    recall  f1-score   support

           0       1.00      0.12      0.21        43
           1       1.00      1.00      1.00     46049

    accuracy                           1.00     46092
   macro avg       1.00      0.56      0.60     46092
weighted avg       1.00      1.00      1.00     46092



In this case, my sensitivity score (true positives / all positives) is 59%.
Earlier, the logistic regression model fit on my more balanced data had a sensitivity of 73 / 76, or about 96%.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!